In [0]:
import json
import pandas as pd
import numpy as np
import os
import sys
import tensorflow as tf
import tensorflow_hub as hub
from time import time
import io
import re
import csv
import matplotlib.pyplot as plt

import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib.ticker import PercentFormatter

from keras.utils import to_categorical
from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_fscore_support,classification_report
from sklearn.preprocessing import LabelEncoder

from tensorflow.keras import backend as K
from tensorflow.keras import layers
from tensorflow.python.keras.layers import Lambda
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.backend import sparse_categorical_crossentropy
from tensorflow.keras.layers import Dense, TimeDistributed
from tensorflow.python.tools import inspect_checkpoint as chkp

from sklearn.model_selection import train_test_split

from datetime import datetime

In [0]:
# !pip install tf-nightly-2.0-preview

# !pip install "tensorflow>=1.7.0"

# !pip install tensorflow-hub


In [2]:
import tensorflow as tf
import tensorflow_hub as hub
print(tf.__version__)

1.14.0


In [3]:
from google.colab import drive


drive.mount('/content/drive/')
__dir__ = '/content/drive/My Drive/Colab Notebooks/'

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [4]:
!pip install bert-tensorflow

In [5]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization


W0803 20:38:53.018659 140093742360448 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/bert/optimization.py:87: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [0]:
# !ls /content/gdrive/My\ Drive/Colab\ Notebooks/custom_module/bert-ner/*.py
# !cat /content/gdrive/My\ Drive/Colab\ Notebooks/custom_module/bert_ner/run_classifier_ner.py
# import sys
# sys.path.append('/content/gdrive/My\ Drive/Colab\ Notebooks/custom_module/bert_ner/')



In [6]:
OUTPUT_DIR = '/content/drive/My Drive/Colab Notebooks/Model_Output/'
tf.io.gfile.makedirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))

***** Model output directory: /content/drive/My Drive/Colab Notebooks/Model_Output/ *****


In [7]:
BERT_MODEL_HUB = 'https://tfhub.dev/google/bert_cased_L-12_H-768_A-12/1'

def create_tokenizer_from_hub_module():
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info =  bert_module(signature='tokenization_info', as_dict=True)
#     print(tokenization_info)
    with tf.compat.v1.Session() as sess:
      vocab_file = sess.run(tokenization_info['vocab_file'])
      
  return bert.tokenization.FullTokenizer(vocab_file=vocab_file)


tokenizer = create_tokenizer_from_hub_module()

W0803 20:38:56.904978 140093742360448 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/bert/tokenization.py:125: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.



In [8]:
tokenizer.tokenize("This here's an example of using the BERT tokenizer")

['this',
 'here',
 "'",
 's',
 'an',
 'example',
 'of',
 'using',
 'the',
 'be',
 '##rt',
 'token',
 '##izer']

In [9]:
def checkPrior(blah):
	if (blah.prior is None or blah.prior is np.NaN) and (blah.prior_pos is None or blah.prior_pos is np.NaN):
		return True
	else:
		return False




def create_onto_data_vectors(data_type):
  data_type_dict = {'test' : '/content/drive/My Drive/Colab Notebooks/data/onto.test.ner',
                   'dev' : '/content/drive/My Drive/Colab Notebooks/data/onto.development.ner',
                   'train' : '/content/drive/My Drive/Colab Notebooks/data/onto.train.ner',
                   'sample': '/content/drive/My Drive/Colab Notebooks/data/onto.train.ner.sample'}
  header = 0
  if 'onto' in data_type_dict[data_type]:
    header = None
  data = pd.read_csv(data_type_dict[data_type], sep="\t",  quoting=csv.QUOTE_NONE, header=header, skip_blank_lines=False, engine='python', names =['token', 'pos', 'tree', 'label'])

  # see if prior row was a newline
  data['prior'] = data.token.shift(1)
  data['prior_pos'] = data.pos.shift(1)
  # drop empty rows
  data = data.loc[~data.token.isnull()]
  data.prior = data.apply(checkPrior, axis=1)
  data['phrase'] = data.prior.cumsum()
  df = data.groupby('phrase').agg({'token':list, 'label': list})

  df['token'] = df['token'].apply(lambda x: ' '.join(x))
  X = df['token'].tolist()
  y = df['label'].tolist()
  return X, y

X, y = create_onto_data_vectors('train')

X_dev, y_dev = create_onto_data_vectors('dev')

X_test, y_test = create_onto_data_vectors('test')



X_train, y_train = create_onto_data_vectors('train')
label_list = list(set([label for target_list in y_train for label in target_list])) + ['[CLS]', '[SEP]']
# print(label_list)
label_map = {label : i for i, label in enumerate(label_list,1)}
print(label_map)
print(f'Length of labels - {len(label_list)}')


X_dev, y_dev = create_onto_data_vectors('dev')

print(len(X_train))
print(len(y_train))
print(y_train[0])
print(X_train[0])

{'B-WORK_OF_ART': 1, 'I-EVENT': 2, 'I-GPE': 3, 'I-TIME': 4, 'I-QUANTITY': 5, 'B-NORP': 6, 'B-PERCENT': 7, 'O': 8, 'I-NORP': 9, 'I-WORK_OF_ART': 10, 'I-FAC': 11, 'I-LANGUAGE': 12, 'B-ORDINAL': 13, 'B-PERSON': 14, 'B-PRODUCT': 15, 'I-LOC': 16, 'B-TIME': 17, 'B-ORG': 18, 'I-DATE': 19, 'I-PERSON': 20, 'B-LANGUAGE': 21, 'B-LAW': 22, 'B-QUANTITY': 23, 'I-PERCENT': 24, 'B-MONEY': 25, 'B-CARDINAL': 26, 'B-DATE': 27, 'I-PRODUCT': 28, 'B-FAC': 29, 'I-MONEY': 30, 'B-EVENT': 31, 'I-ORG': 32, 'B-LOC': 33, 'I-LAW': 34, 'B-GPE': 35, 'I-ORDINAL': 36, 'I-CARDINAL': 37, '[CLS]': 38, '[SEP]': 39}
Length of labels - 39
115812
115812
['O', 'O', 'O', 'O', 'O']
What kind of memory ?


In [0]:
# file = pd.ExcelFile('/content/drive/My Drive/Colab Notebooks/data/test_data_no_title.xlsx')

# df = file.parse('Sheet2', header=None)
# df_words = df[0]
# df_ner = df[1]

# df_words.loc[df_words.isnull()] = df_words.loc[df_words.isnull()].apply(lambda x: 'BREAK')
# df_ner.loc[df_ner.isnull()] = df_ner.loc[df_ner.isnull()].apply(lambda x: 'BREAK')


# words_list = df_words.tolist()
# ner_list = df_ner.tolist()

# word_seq = []
# word_sentences = []
# for count, word in enumerate(words_list):
#     if word != 'BREAK':
#         word_seq.append(str(word))
        
#     else:
#         word_sentences.append(' '.join(word_seq))
#         word_seq = []
#         continue
# print(f'total tokens - {count}')

# ner_seq = []
# ner_sentences = []
# for count_tags, ner in enumerate(ner_list):
#     if ner != 'BREAK':
#         ner_seq.append(ner)
#     else:
#         ner_sentences.append(ner_seq)
#         ner_seq = []
#         continue

# print(f'Total tags - {count_tags}')

# #print(word_sentences[0])    
# #print(ner_sentences[0])    

# X_test, y_test = word_sentences, ner_sentences

# print(len(X_test))
# print(len(y_test))

# print(X_test[0])
# print(y_test[0])


In [0]:
FILE_PATH = '/content/drive/My Drive/Colab Notebooks/data/bios-tagged-final-flat.csv'

header = 0
if 'onto' in FILE_PATH:
	header = None
data = pd.read_csv(FILE_PATH, sep="\t",  quoting=csv.QUOTE_NONE, header=header, skip_blank_lines=False, engine='python', names =['token', 'pos', 'tree', 'label'])

# see if prior row was a newline
data['prior'] = data.token.shift(1)
data['prior_pos'] = data.pos.shift(1)
# drop empty rows
data = data.loc[~data.token.isnull()]
data.prior = data.apply(checkPrior, axis=1)
data['phrase'] = data.prior.cumsum()

#aggregate by sentence
bios_df = data.groupby('phrase').agg({'token':list, 'label': list})
bios_df['token'] = bios_df['token'].apply(lambda x: ' '.join(x))

X_bios = bios_df['token'].tolist()
y_bios = bios_df['label'].tolist()

y_bios_no_title = [['O' if label in ['B-TITLE','U-TITLE','L-TITLE','I-TITLE'] else label for label in seq] for seq in y_bios]

In [11]:
print(X_bios[0])
print(y_bios_no_title[0])

Jeffrey P. Bezos , age 55 , has been Chairman of the Board since founding the Company in 1994 and Chief Executive Officer since May 1996 .
['B-PERSON', 'I-PERSON', 'I-PERSON', 'O', 'B-DATE', 'I-DATE', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'O', 'O', 'O', 'B-ORG', 'O', 'B-DATE', 'O', 'O', 'O', 'O', 'O', 'B-DATE', 'I-DATE', 'O']


In [12]:
#transfer sentences to txt file, one ssentence per line, for ELMO encoder

# os.chdir('/content/drive/My Drive/Colab Notebooks/data/')

# test_list = [['a', 'b', 'c', '.'],['1','2']]

# with open('test.txt', 'w+') as file:
#   for line in test_list:
#     file.write(' '.join(line) + '\n')

# with open('X_train-MICRO.txt', 'w') as file:
#   for indx, seq in enumerate(X_train):
#     if len(seq) > 0 and indx < 100:
#       file.write(seq + '\n')


label_map = {label : i for i, label in enumerate(label_list,1)}
print(label_map)

{'B-WORK_OF_ART': 1, 'I-EVENT': 2, 'I-GPE': 3, 'I-TIME': 4, 'I-QUANTITY': 5, 'B-NORP': 6, 'B-PERCENT': 7, 'O': 8, 'I-NORP': 9, 'I-WORK_OF_ART': 10, 'I-FAC': 11, 'I-LANGUAGE': 12, 'B-ORDINAL': 13, 'B-PERSON': 14, 'B-PRODUCT': 15, 'I-LOC': 16, 'B-TIME': 17, 'B-ORG': 18, 'I-DATE': 19, 'I-PERSON': 20, 'B-LANGUAGE': 21, 'B-LAW': 22, 'B-QUANTITY': 23, 'I-PERCENT': 24, 'B-MONEY': 25, 'B-CARDINAL': 26, 'B-DATE': 27, 'I-PRODUCT': 28, 'B-FAC': 29, 'I-MONEY': 30, 'B-EVENT': 31, 'I-ORG': 32, 'B-LOC': 33, 'I-LAW': 34, 'B-GPE': 35, 'I-ORDINAL': 36, 'I-CARDINAL': 37, '[CLS]': 38, '[SEP]': 39}


In [13]:
print(X_train[0])

What kind of memory ?


In [0]:


def conv_to_df(X, y):

  df = pd.DataFrame([X,y]).T
  df.columns = ['X', 'y']
#   print (type(df.loc[0, 'X']))
#   print (type(df.loc[0, 'y']))
#   df['y'] = df['y'].str.strip('[]')
#   print(df.head())
  return df

train = conv_to_df(X_train, y_train)
dev = conv_to_df(X_dev, y_dev)
test = conv_to_df(X_test, y_test)
bios = conv_to_df(X_bios, y_bios_no_title)


In [0]:
# train_to_txt = train['X']

# dev_to_txt = dev['X']

# # train_to_txt.to_csv('/content/drive/My Drive/Colab Notebooks/data/X_train.csv', header=None, index=None,
# #                     quoting=csv.QUOTE_MINIMAL, na_rep='', escapechar='\\')
# os.chdir('/content/drive/My Drive/Colab Notebooks/data/')

# # pd.set_option("display.max_colwidth", 256)

# with open('test.txt', 'w+') as f:
#     train_to_txt.to_string(f, index=False)

# # dev_to_txt.to_csv('/content/drive/My Drive/Colab Notebooks/data/X_dev.txt', header=None, index=None, quoting=csv.QUOTE_NONE)

# dev_to_txt.head()

In [0]:
import logging
logger = logging.getLogger(__name__)

class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, input_mask, segment_ids, label_id, valid_ids=None, label_mask=None):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_id = label_id
        self.valid_ids = valid_ids
        self.label_mask = label_mask

def convert_examples_to_features(examples, label_list, max_seq_length, tokenizer):
    """Loads a data file into a list of `InputBatch`s."""

    label_map = {label : i for i, label in enumerate(label_list,1)}
    
    features = []
    for (ex_index,example) in enumerate(examples):
        textlist = example.text_a.split(' ')
        labellist = example.label
        if len(textlist) != len(labellist):
          continue
        else:
          tokens = []
          labels = []
          valid = []
          label_mask = []
          for i, word in enumerate(textlist): 
              token = tokenizer.tokenize(word)
              tokens.extend(token)

              label_1 = labellist[i]
              for m in range(len(token)):
                  if m == 0:
                      labels.append(label_1)
                      valid.append(1)
                      label_mask.append(1)
                  else:
                      valid.append(0)
          if len(tokens) >= max_seq_length - 1:
              tokens = tokens[0:(max_seq_length - 2)]
              labels = labels[0:(max_seq_length - 2)]
              valid = valid[0:(max_seq_length - 2)]
              label_mask = label_mask[0:(max_seq_length - 2)]
          ntokens = []
          segment_ids = []
          label_ids = []
          ntokens.append("[CLS]")
          segment_ids.append(0)
          valid.insert(0,1)
          label_mask.insert(0,1)
          label_ids.append(label_map["[CLS]"])
          for i, token in enumerate(tokens):
              ntokens.append(token)
              segment_ids.append(0)
              if len(labels) > i:
                  label_ids.append(label_map[labels[i]])
          ntokens.append("[SEP]")
          segment_ids.append(0)
          valid.append(1)
          label_mask.append(1)
          label_ids.append(label_map["[SEP]"])
          input_ids = tokenizer.convert_tokens_to_ids(ntokens)
          input_mask = [1] * len(input_ids)
          label_mask = [1] * len(label_ids)
          while len(input_ids) < max_seq_length:
              input_ids.append(0)
              input_mask.append(0)
              segment_ids.append(0)
              label_ids.append(0)
              valid.append(1)
              label_mask.append(0)
          while len(label_ids) < max_seq_length:
              label_ids.append(0)
              label_mask.append(0)
          assert len(input_ids) == max_seq_length
          assert len(input_mask) == max_seq_length
          assert len(segment_ids) == max_seq_length
          assert len(label_ids) == max_seq_length
          assert len(valid) == max_seq_length
          assert len(label_mask) == max_seq_length

          if ex_index < 5:
              logger.info("*** Example ***")
              logger.info("guid: %s" % (example.guid))
              logger.info("tokens: %s" % " ".join(
                      [str(x) for x in tokens]))
              logger.info("input_ids: %s" % " ".join([str(x) for x in input_ids]))
              logger.info("input_mask: %s" % " ".join([str(x) for x in input_mask]))
              logger.info(
                      "segment_ids: %s" % " ".join([str(x) for x in segment_ids]))
#               logger.info("label: %s (id = %d)" % (example.label, label_ids))

          features.append(
                  InputFeatures(input_ids=input_ids,
                                input_mask=input_mask,
                                segment_ids=segment_ids,
                                label_id=label_ids,
                                valid_ids=valid,
                                label_mask=label_mask))
    return features

In [0]:
train_InputExamples = train.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x['X'], 
                                                                   text_b = None, 
                                                                   label = x['y']), axis = 1)

dev_InputExamples = dev.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x['X'], 
                                                                   text_b = None, 
                                                                   label = x['y']), axis = 1)

test_InputExamples = test.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x['X'], 
                                                                   text_b = None, 
                                                                   label = x['y']), axis = 1)

bios_InputExamples = bios.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x['X'], 
                                                                   text_b = None, 
                                                                   label = x['y']), axis = 1)

In [0]:
MAX_SEQ_LENGTH = 56
max_length = MAX_SEQ_LENGTH




In [0]:
# Convert our train and test features to InputFeatures that BERT understands.
train_features = convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

In [0]:
dev_features = convert_examples_to_features(dev_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

In [0]:
test_features = convert_examples_to_features(test_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

In [0]:
bios_features = convert_examples_to_features(bios_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

In [0]:
trainSentence_ids = []
trainMasks = []
trainSequence_ids = []

devSentence_ids = []
devMasks = []
devSequence_ids = []

testSentence_ids = []
testMasks = []
testSequence_ids = []

biosSentence_ids = []
biosMasks = []
biosSequence_ids = []


nerLabels_train =[]
nerLabels_dev = []
nerLabels_test = []
nerLabels_bios = []



bert_inputs_train = np.array([[f.input_ids for f in train_features], [f.input_mask for f in train_features],[f.segment_ids for f in train_features]])
labels_train = [f.label_id for f in train_features]
num_sentences_train = len([f.input_ids for f in train_features])
# print(num_sentences)
for example in range(num_sentences_train):
    trainSentence_ids.append(bert_inputs_train[0][example])
    trainMasks.append(bert_inputs_train[1][example])
    trainSequence_ids.append(bert_inputs_train[2][example])
    nerLabels_train.append(labels_train[example])

    
# labels_train = [to_categorical(f.label_id) for f in train_features]


bert_inputs_dev = [[f.input_ids for f in dev_features], [f.input_mask for f in dev_features],[f.segment_ids for f in dev_features]]
labels_dev = [f.label_id for f in dev_features]
num_sentences_dev = len([f.input_ids for f in dev_features])
for example in range(num_sentences_dev):
    devSentence_ids.append(bert_inputs_dev[0][example])
    devMasks.append(bert_inputs_dev[1][example])
    devSequence_ids.append(bert_inputs_dev[2][example])
    nerLabels_dev.append(labels_dev[example])

    
bert_inputs_test = np.array([[f.input_ids for f in test_features], [f.input_mask for f in test_features],[f.segment_ids for f in test_features]])
labels_test = [f.label_id for f in test_features]
num_sentences_test = len([f.input_ids for f in test_features])
# print(num_sentences)
for example in range(num_sentences_test):
    testSentence_ids.append(bert_inputs_test[0][example])
    testMasks.append(bert_inputs_test[1][example])
    testSequence_ids.append(bert_inputs_test[2][example])
    nerLabels_test.append(labels_test[example])    
    
    
bert_inputs_bios = np.array([[f.input_ids for f in bios_features], [f.input_mask for f in bios_features],[f.segment_ids for f in bios_features]])
labels_bios = [f.label_id for f in bios_features]
num_sentences_bios = len([f.input_ids for f in bios_features])
# print(num_sentences)
for example in range(num_sentences_bios):
    biosSentence_ids.append(bert_inputs_bios[0][example])
    biosMasks.append(bert_inputs_bios[1][example])
    biosSequence_ids.append(bert_inputs_bios[2][example])
    nerLabels_bios.append(labels_bios[example])    
        
    
X_train = np.array([trainSentence_ids,trainMasks,trainSequence_ids])
X_dev = np.array([devSentence_ids,devMasks,devSequence_ids])
X_test = np.array([testSentence_ids,testMasks,testSequence_ids])
X_bios = np.array([biosSentence_ids,biosMasks,biosSequence_ids])


nerLabels_train = np.array(nerLabels_train)
nerLabels_dev = np.array(nerLabels_dev)
nerLabels_test = np.array(nerLabels_test)
nerLabels_bios = np.array(nerLabels_bios)





# labels_dev = [to_categorical(f.label_id) for f in dev_features]

#                                 input_ids=input_ids,
#                                 input_mask=input_mask,
#                                 segment_ids=segment_ids,
#                                 label_id=label_ids,
#                                 valid_ids=valid,
#                                 label_mask=label_mask))

In [24]:
X_test.shape

(3, 12217, 56)

In [0]:
k_start = 0
k_end = -1

if k_end == -1:
    k_end_train = X_train[0].shape[0]
    k_end_dev = X_dev[0].shape[0]
    k_end_test = X_test[0].shape[0]
else:
    k_end_train = k_end_dev = k_end
    


bert_inputs_train_k = [X_train[0][k_start:k_end_train], X_train[1][k_start:k_end_train], 
                       X_train[2][k_start:k_end_train]]
bert_inputs_dev_k = [X_dev[0][k_start:k_end_dev], X_dev[1][k_start:k_end_dev], 
                      X_dev[2][k_start:k_end_dev]]
bert_inputs_test_k = [X_test[0][k_start:k_end_test], X_test[1][k_start:k_end_test], 
                      X_test[2][k_start:k_end_test]]
bert_inputs_bios_k = [X_bios[0][k_start:k_end_test], X_bios[1][k_start:k_end_test], 
                      X_bios[2][k_start:k_end_test]]



labels_train_k = nerLabels_train[k_start:k_end_train]
labels_dev_k = nerLabels_dev[k_start:k_end_dev]
labels_test_k = nerLabels_test[k_start:k_end_test]
labels_bios_k = nerLabels_bios[k_start:k_end_test]

In [26]:
for item in bert_inputs_train_k:
  print(item.shape)

(115812, 56)
(115812, 56)
(115812, 56)


In [0]:
# bert_inputs_train[0][0]

In [0]:
# # check_labels = [f.label_id for f in train_features]
# print(x_label_mask[0])
# # print(labels_train[0])

In [0]:
# print(x_valid_ids[0])

In [0]:
class BertLayer(tf.keras.layers.Layer):
    """
    Create BERT layer, following https://towardsdatascience.com/bert-in-keras-with-tensorflow-hub-76bcbc9417b
    init:  initialize layer. Specify various parameters regarding output types and dimensions. Very important is
           to set the number of trainable layers.
    build: build the layer based on parameters
    call:  call the BERT layer within a model
    """
    
    def __init__(
        self,
        n_fine_tune_layers=10,
        pooling="sequence",
        bert_url="https://tfhub.dev/google/bert_cased_L-12_H-768_A-12/1",
        **kwargs,
    ):
        self.n_fine_tune_layers = n_fine_tune_layers
        self.trainable = True
        self.output_size = 768
        self.pooling = pooling
        self.bert_url = bert_url

        super(BertLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.bert = hub.Module(
            self.bert_url, trainable=self.trainable, name=f"{self.name}_module"
        )

        # Remove unused layers
        trainable_vars = self.bert.variables
        trainable_vars = [
                var
                for var in trainable_vars
                if not "/cls/" in var.name and not "/pooler/" in var.name
            ]
        trainable_layers = []


        # Select how many layers to fine tune
        for i in range(self.n_fine_tune_layers):
            trainable_layers.append(f"encoder/layer_{str(11 - i)}")

        # Update trainable vars to contain only the specified layers
        trainable_vars = [
            var
            for var in trainable_vars
            if any([l in var.name for l in trainable_layers])
        ]

        # Add to trainable weights
        for var in trainable_vars:
            self._trainable_weights.append(var)

        for var in self.bert.variables:
            if var not in self._trainable_weights:
                self._non_trainable_weights.append(var)

        super(BertLayer, self).build(input_shape)

    def call(self, inputs):
        inputs = [K.cast(x, dtype="int32") for x in inputs]
        input_ids, input_mask, segment_ids = inputs
        bert_inputs = dict(
            input_ids=input_ids, input_mask=input_mask, segment_ids=segment_ids
        )
        result = self.bert(inputs=bert_inputs, signature="tokens", as_dict=True)[
                "sequence_output"
            ]

        mul_mask = lambda x, m: x * tf.expand_dims(m, axis=-1)

        return result

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_size)

In [0]:


def variable_summaries(var):
    """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.histogram('histogram', var)



In [0]:


def custom_loss(y_true, y_pred):
    """
    calculate loss function explicitly, filtering out 'extra inserted labels'
    
    y_true: Shape: (batch x (max_length + 1) )
    y_pred: predictions. Shape: (batch x x (max_length + 1) x num_distinct_ner_tokens ) 
    
    returns:  cost
    """

    #get labels and predictions
    
    y_label = tf.reshape(tf.layers.Flatten()(tf.cast(y_true, tf.int32)),[-1])
#     print(y_label.eval())
    mask = (y_label < 38)   # This mask is used to remove all tokens that do not correspond to the original base text.

    y_label_masked = tf.boolean_mask(y_label, mask)  # mask the labels
    
    y_flat_pred = tf.reshape(tf.layers.Flatten()(tf.cast(y_pred, tf.float32)),[-1, 39])
#     print(y_flat_pred.eval())
    y_flat_pred_masked = tf.boolean_mask(y_flat_pred, mask) # mask the predictions
    
    return tf.reduce_mean(sparse_categorical_crossentropy(y_label_masked, y_flat_pred_masked,from_logits=False ))

In [0]:
def custom_acc_orig_tokens(y_true, y_pred):
    """
    calculate loss dfunction filtering out also the newly inserted labels
    
    y_true: Shape: (batch x (max_length) )
    y_pred: predictions. Shape: (batch x x (max_length + 1) x num_distinct_ner_tokens ) 
    
    returns: accuracy
    """

    #get labels and predictions
    
    y_label = tf.reshape(tf.layers.Flatten()(tf.cast(y_true, tf.int64)),[-1])
    
    mask = (y_label < 38)
    y_label_masked = tf.boolean_mask(y_label, mask)
    
    y_predicted = tf.math.argmax(input = tf.reshape(tf.layers.Flatten()(tf.cast(y_pred, tf.float64)),\
                                                    [-1, 39]), axis=1)
    
    y_predicted_masked = tf.boolean_mask(y_predicted, mask)

    return tf.reduce_mean(tf.cast(tf.equal(y_predicted_masked,y_label_masked) , dtype=tf.float64))

In [0]:
def custom_acc_orig_non_other_tokens(y_true, y_pred):
    """
    calculate loss dfunction explicitly filtering out also the 'Other'- labels
    
    y_true: Shape: (batch x (max_length) )
    y_pred: predictions. Shape: (batch x x (max_length + 1) x num_distinct_ner_tokens ) 
    
    returns: accuracy
    """

    #get labels and predictions
    
    y_label = tf.reshape(tf.layers.Flatten()(tf.cast(y_true, tf.int64)),[-1])
    
    mask = (y_label < 38)
    y_label_masked = tf.boolean_mask(y_label, mask)
    
    y_predicted = tf.math.argmax(input = tf.reshape(tf.layers.Flatten()(tf.cast(y_pred, tf.float64)),\
                                                    [-1, 39]), axis=1)
    
    y_predicted_masked = tf.boolean_mask(y_predicted, mask)

    return tf.reduce_mean(tf.cast(tf.equal(y_predicted_masked,y_label_masked) , dtype=tf.float64))

In [0]:
adam_customized = tf.keras.optimizers.Adam(lr=0.001, beta_1=0.91, beta_2=0.999, epsilon=None, decay=0.1, amsgrad=False)

In [0]:
# label_list = {'I-EVENT': 1, 'B-LOC': 2, 'I-MONEY': 3, 'I-FAC': 4, 'B-FAC': 5, 'I-LANGUAGE': 6, 'B-DATE': 7, 'I-WORK_OF_ART': 8, 'B-PRODUCT': 9, 'B-TIME': 10, 'I-GPE': 11, 'B-ORG': 12, 'B-CARDINAL': 13, 'I-PERSON': 14, 'B-GPE': 15, 'B-QUANTITY': 16, 'I-NORP': 17, 'I-ORDINAL': 18, 'B-PERSON': 19, 'B-WORK_OF_ART': 20, 'I-PERCENT': 21, 'I-ORG': 22, 'B-MONEY': 23, 'I-PRODUCT': 24, 'I-QUANTITY': 25, 'I-LAW': 26, 'B-EVENT': 27, 'I-LOC': 28, 'B-ORDINAL': 29, 'B-LANGUAGE': 30, 'B-PERCENT': 31, 'O': 32, 'I-DATE': 33, 'I-TIME': 34, 'B-LAW': 35, 'I-CARDINAL': 36, 'B-NORP': 37, '[CLS]': 38, '[SEP]': 39}

# y_true = tf.constant([[0],[0]])

# y_pred = tf.constant([
#     [0.6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,.4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
#     [0.6,0.4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
# ])

# sess = tf.InteractiveSession()
# sess.run(tf.global_variables_initializer())

# print(custom_loss(y_true, y_pred).eval())

# sess.close()

In [0]:


def ner_model(max_input_length, train_layers, optimizer):
    """
    Implementation of NER model
    
    variables:
        max_input_length: number of tokens (max_length + 1)
        train_layers: number of layers to be retrained
        optimizer: optimizer to be used
    
    returns: model
    """
    
    in_id = tf.keras.layers.Input(shape=(max_length,), name="input_ids")
    in_mask = tf.keras.layers.Input(shape=(max_length,), name="input_masks")
    in_segment = tf.keras.layers.Input(shape=(max_length), name="segment_ids")
    
    
    bert_inputs = [in_id, in_mask, in_segment]
    
    bert_sequence = BertLayer(n_fine_tune_layers=train_layers)(bert_inputs)
    
    print(bert_sequence)
    
    dense = tf.keras.layers.Dense(256, activation='relu', name='dense')(bert_sequence)
    
#     dense = tf.keras.layers.Dropout(rate=0.2)(dense)
    
    pred = tf.keras.layers.Dense(39, activation='softmax', name='ner')(dense)
     
    print('pred: ', pred)
    
    ## Prepare for multipe loss functions, although not used here
    
    losses = {
        "ner": custom_loss,
        }
    lossWeights = {"ner": 1.0
                  }
    
    model = tf.keras.models.Model(inputs=bert_inputs, outputs=pred)

#     model.compile(loss=losses, optimizer=optimizer, metrics=[custom_acc_orig_tokens, 
#                                                           custom_acc_orig_non_other_tokens])
    
    model.compile(loss= losses, optimizer=optimizer, metrics=[custom_acc_orig_tokens, 
                                                          custom_acc_orig_non_other_tokens])
    model.summary()
    
    return model

def initialize_vars(sess):
    sess.run(tf.local_variables_initializer())
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    K.set_session(sess)



In [40]:
#Start session

sess = tf.Session()

model = ner_model(max_length , train_layers=10, optimizer = adam_customized)

# Instantiate variables
initialize_vars(sess)



# tensorboard = TensorBoard(log_dir="logs/{}".format(time()))

model.fit(
    bert_inputs_train_k, 
    {"ner":labels_train_k},
    validation_data=(bert_inputs_dev_k, {"ner":labels_dev_k}),
    epochs=3,
    batch_size=50 #,
    #callbacks=[tensorboard]
)

Tensor("bert_layer_1/bert_layer_1_module_apply_tokens/bert/encoder/Reshape_13:0", shape=(?, ?, 768), dtype=float32)
pred:  Tensor("ner_1/truediv:0", shape=(?, ?, 39), dtype=float32)
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 56)]         0                                            
__________________________________________________________________________________________________
input_masks (InputLayer)        [(None, 56)]         0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 56)]         0                                            
__________________________________________________________________________________________________
bert_laye

In [0]:
with open(OUTPUT_DIR + 'bert_bio_architecture.json', 'w') as f:
    f.write(model.to_json())

model.save_weights(OUTPUT_DIR + 'bert_bio_model_weights.h5')


In [53]:
score = model.evaluate(bert_inputs_test_k, labels_test_k)
print(f'Model loss: {score}')
#     bert_inputs_train_k, 
#     {"ner":labels_train_k}

12217/12217 [==============================] - 95s 8ms/sample - loss: 0.2025 - custom_acc_orig_tokens: 0.9588 - custom_acc_orig_non_other_tokens: 0.9588
Model loss: [0.20248660124033954, 0.958773, 0.958773]


In [0]:
y_pred = model.predict(bert_inputs_test_k).argmax(2)
yh = labels_test_k
fyh_test, fpr_test = y_pred.flatten()

In [0]:
# accuracy_score(fyh_test, fpr_test)

In [54]:
score = model.evaluate(bert_inputs_bios_k, labels_bios_k)
print(f'Model loss: {score}')

708/708 [==============================] - 6s 8ms/sample - loss: 0.5804 - custom_acc_orig_tokens: 0.8585 - custom_acc_orig_non_other_tokens: 0.8585
Model loss: [0.5804318546575341, 0.85846925, 0.85846925]


In [46]:
print('Training accuracy Onto TEST : ', accuracy_score(fyh_test, fpr_test))

Training accuracy Onto TEST :  0.9245109273962512


In [0]:

print(classification_report(fyh_test, fpr_test))

In [0]:
y_pred = model.predict(bert_inputs_bios_k).argmax(2)
yh = labels_bios_k
fyh_bios, fpr_bios = y_pred.flatten(), yh.flatten()

In [49]:
print('Training accuracy DEF14A : ', accuracy_score(y_pred.flatten(), yh.flatten()))
# print('Training confusion matrix:')
cm = confusion_matrix(fyh_test, fpr_test)

Training accuracy DEF14A :  0.8252370863599677


In [0]:
print(classification_report(fyh_bios, fpr_bios))